In [5]:
# Import library
import pandas  as pd #Data manipulation
import numpy as np #Data manipulation
import matplotlib.pyplot as plt # Visualization
import seaborn as sns #Visualization

In [7]:
classification_data = pd.read_csv('classification.csv', sep = ',')
classification_data.head()

,x1,x2,y
0,4.714274,0.989079,2
1,1.661432,6.846034,2
2,1.360534,3.359866,2
3,1.720238,2.021597,2
4,4.959555,8.563527,1


In [75]:
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
import random

In [122]:
### import random
validate = np.ndarray(shape = (10, 30))
# validate_std  = np.ndarray(shape = (10, 30))
accurary = np.ndarray(shape = (10, 30))


for i in range(10):
    random.seed(i)
    # shuffle data for 10 time
#!!!!Don't delete frac = 1
### For information about frac = 1, see this website: https://stackoverflow.com/questions/29576430/shuffle-dataframe-rows
    shuffeled_data =classification_data.sample(frac=1).reset_index(drop = True)
    splited_data = [shuffeled_data.loc[:20]]
    splited_data.append(shuffeled_data.loc[20:])
    #check the splited daata
    
    ## first get the original 20 points of the 
    train_X = splited_data[0][["x1","x2"]].values
    train_Y = splited_data[0]["y"].values
    test_X = splited_data[1][["x1","x2"]].values
    test_Y = splited_data[1]["y"].values

#     print(test_X,type(test_Y))
    clf = svm.SVC(kernel='linear').fit(train_X,train_Y)

    scores = cross_val_score(clf,train_X,train_Y,scoring='accuracy', cv = 5)

#     print(scores, "scores")
    ###################### Question: should I just do the average at here???????////
    validate[i,0] = np.mean(scores)

    y_pred = clf.predict(test_X)

    accurary[i,0] = accuracy_score(test_Y, y_pred)

#     print("validate",validate,"accuracu",accurary, i, j)
#     print('shape of textX ',test_X[0])
    for j in range(1,30):
        train_X = np.concatenate((train_X, [test_X[0]]))

        train_Y = np.concatenate((train_Y,[test_Y[0]]))
#         print(train_Y )
#         print(train_X,train_Y)
        test_X = np.delete(test_X, 0,0)
        test_Y = np.delete(test_Y, 0,0)
#         print(test_Y, len(test_X))
        clf = svm.SVC(kernel='linear').fit(train_X,train_Y)

        scores = cross_val_score(clf,train_X,train_Y,scoring='accuracy', cv = 5)

        validate[i,j] = np.mean(scores)
        
        y_pred = clf.predict(test_X)
        accurary[i,j] = accuracy_score(test_Y, y_pred)


In [123]:
print(validate, 'acc',accurary)

[[0.57       0.58       0.57       0.59       0.6        0.73333333
  0.78666667 0.72666667 0.76       0.7        0.70952381 0.6952381
  0.7047619  0.7047619  0.71428571 0.725      0.71071429 0.71428571
  0.74642857 0.75       0.73888889 0.77222222 0.77222222 0.75
  0.71111111 0.74       0.70666667 0.70888889 0.70888889 0.7       ]
 [0.86       0.78       0.78       0.8        0.72       0.76666667
  0.9        0.82666667 0.82666667 0.83333333 0.84285714 0.85238095
  0.79047619 0.73333333 0.68571429 0.69285714 0.7        0.67857143
  0.65714286 0.725      0.75555556 0.68888889 0.74166667 0.72222222
  0.68888889 0.71555556 0.72       0.70444444 0.67333333 0.7       ]
 [0.82       0.86       0.87       0.88       0.84       0.84666667
  0.85333333 0.86       0.79333333 0.8        0.8047619  0.81428571
  0.85238095 0.85714286 0.85714286 0.86071429 0.89285714 0.84285714
  0.87142857 0.875      0.87777778 0.88055556 0.86111111 0.86388889
  0.86666667 0.86888889 0.87333333 0.87555556 0.87777

In [124]:
#### part II, using uncertainty sampling
validate = np.ndarray(shape = (10, 30))
# validate_std  = np.ndarray(shape = (10, 30))
accurary = np.ndarray(shape = (10, 30))
for i in range(10):
    random.seed(i)
    shuffeled_data =classification_data.sample(frac=1).reset_index(drop = True)
    ### split data set
    splited_data = [shuffeled_data.loc[:20]]
    splited_data.append(shuffeled_data.loc[20:])
    ### Transform dataframe into np array
    train_X = splited_data[0][["x1","x2"]].values
    train_Y = splited_data[0]["y"].values
    test_X = splited_data[1][["x1","x2"]].values
    test_Y = splited_data[1]["y"].values
    ### calculate
    clf = svm.SVC(kernel='linear',probability=True).fit(train_X,train_Y)
    scores = cross_val_score(clf,train_X,train_Y,scoring='accuracy', cv = 5)
#     print(scores, "scores")
    ###################### Question: should I just do the average at here???????////
    validate[i,0] = np.mean(scores)
    y_pred = clf.predict(test_X)
    decision_sample =test_X[np.argmin(np.abs(clf.decision_function(test_X)))] 
    print(decision_sample, "decision_sample")
#     print(y_pred)
    accurary[i,0] = accuracy_score(test_Y, y_pred)
#     print("validate",validate,"accuracu",accurary, i, j)
#     print('shape of textX ',test_X[0])
    for j in range(1,30):
        train_X = np.concatenate((train_X, [test_X[0]]))

        train_Y = np.concatenate((train_Y,[test_Y[0]]))
#         print(train_Y )
#         print(train_X,train_Y)
        test_X = np.delete(test_X, 0,0)
        test_Y = np.delete(test_Y, 0,0)
#         print(test_Y, len(test_X))
        clf = svm.SVC(kernel='linear').fit(train_X,train_Y)

        scores = cross_val_score(clf,train_X,train_Y,scoring='accuracy', cv = 5)

        validate[i,j] = np.mean(scores)
        
        y_pred = clf.predict(test_X)
        accurary[i,j] = accuracy_score(test_Y, y_pred)

[5.31272374 2.6233942 ] decision_sample
[0.37026921 7.95533397] decision_sample
[3.92531938 4.06247948] decision_sample
[9.96619419 9.53116991] decision_sample
[3.80381738 6.02428607] decision_sample
[6.16207264 3.01948651] decision_sample
[6.01026158 9.16495206] decision_sample
[3.80381738 6.02428607] decision_sample
[2.42063567 6.26646876] decision_sample


C:\Users\tiant\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
C:\Users\tiant\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
C:\Users\tiant\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


[0.17597972 8.40114465] decision_sample


In [125]:
print(validate, 'acc',accurary)

[[0.96       0.96       0.87       0.87       0.88       0.77333333
  0.74       0.74666667 0.76       0.76666667 0.77619048 0.78571429
  0.75238095 0.7952381  0.8        0.80714286 0.78571429 0.76428571
  0.79642857 0.825      0.82777778 0.80555556 0.78888889 0.79166667
  0.86666667 0.82444444 0.80444444 0.78888889 0.75555556 0.76      ]
 [0.67       0.68       0.65       0.67       0.68       0.69333333
  0.74       0.75333333 0.76       0.76666667 0.77619048 0.68571429
  0.76190476 0.85714286 0.74285714 0.86071429 0.81071429 0.73571429
  0.71785714 0.725      0.73055556 0.73611111 0.73888889 0.75277778
  0.73333333 0.74       0.72444444 0.74888889 0.75777778 0.76      ]
 [0.62       0.59       0.7        0.63       0.56       0.57333333
  0.67333333 0.68666667 0.66       0.63333333 0.64285714 0.66190476
  0.67142857 0.64761905 0.62857143 0.66785714 0.70714286 0.63571429
  0.71785714 0.675      0.63333333 0.69166667 0.725      0.73333333
  0.75555556 0.76       0.74666667 0.75333333 